In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import bz2
import zipfile
import locale
from os import path

In [2]:
def get_replay(match_id):
    #check if replay downloaded
    
    
    url = "https://api.opendota.com/api/replays?match_id={}".format(match_id)
    replay_info = requests.get(url)
    
    #get salt and cluster from opendota
    url = "https://api.opendota.com/api/replays?match_id={}".format(match_id)
    raw_json = requests.get(url)
    replay_info = json.loads(raw_json.text)[0]
    
    #download from valve servers
    url = "http://replay{}.valve.net/570/{}_{}.dem.bz2".format(replay_info['cluster'], replay_info['match_id'], replay_info['replay_salt'])
    replay = requests.get(url, allow_redirects=True)
    #with open('{}.bz2'.format(match_id), 'wb') as f:
    #    f.write(replay.content)
    decompressed = bz2.decompress(replay.content)
    with open('replays/{}.dem'.format(replay_info['match_id']), 'wb') as f:
        f.write(decompressed)

In [3]:
def update():
    r = requests.get("http://dota2protracker.com")
    soup = BeautifulSoup(r.text)
    raw_links = soup.findAll('a', href=re.compile('dotabuff'))
    match_ids = []
    for link in raw_links:
        match_id = re.search(r"\d{10,}", str(link))[0]
        match_ids.append(match_id)
    for match_id in match_ids:
        if not path.exists(match_id):
            get_replay(match_id)

In [4]:
update()